In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

In [2]:
# 한글 폰트 경로 설정
font_path = "C:\\Users\\kimmi\\ExJupyterNB\\NanumGothicBold.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [3]:
# 역 리스트
station_list = ['가락시장', '강동구청', '남위례', '남한산성입구(성남법원.검찰청)', '단대오거리', 
                '모란', '몽촌토성(평화의문)', '문정', '복정', '산성', '석촌', '송파', '수진', 
                '신흥', '암사', '잠실(송파구청)', '장지', '천호(풍납토성)']


In [4]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data) - look_back):
        dataX.append(signal_data[i:(i + look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 10

In [6]:
# 각 역에 대한 모델 저장 및 학습
for station_name in station_list:
    # 데이터 불러오기
    data = pd.read_csv(f'{station_name}2.csv', header=None)
    data = data.values.astype('float32')
    data = data.reshape(len(data), 1)

    # 0부터 1로 스케일링
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data)

    train_size = int(len(data) * 0.80)
    test_size = len(data) - train_size

    train = data[0:train_size]
    test = data[train_size:len(data)]
    x_train, y_train = create_dataset(train, look_back)
    x_test, y_test = create_dataset(test, look_back)

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # 모델 정의
    model = Sequential()
    model.add(LSTM(128, input_shape=(None, 1), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(128, input_shape=(None, 1)))
    model.add(Dropout(0.3))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='rmsprop')
    model.summary()
    
    # 모델 학습
    hist = model.fit(x_train, y_train, epochs=10, batch_size=16)
    
    # 모델 저장 (JSON + 가중치)
    model_json = model.to_json()
    with open('model2.json', 'w') as json_file:
       json_file.write(model_json)
    model.save_weights('model2_weights.h5')

    # 모델 예측
    #p = model.predict(x_test)

    # 시각화
    #plt.plot(y_test, label='Actual')
    #plt.plot(p, label='Predicted')
    #plt.legend()
    #plt.title(f'Model Performance for {station_name}')
    #plt.show()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, None, 128)         66560     
                                                                 
 dropout_2 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 198273 (774.50 KB)
Trainable params: 198273 (774.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
821

 lstm_11 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 198273 (774.50 KB)
Trainable params: 198273 (774.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
821/821 [==============================] - 15s 13ms/step - loss: 0.0101
Epoch 2/10
821/821 [==============================] - 11s 13ms/step - loss: 0.0064
Epoch 3/10
821/821 [==============================] - 11s 13ms/step - loss: 0.0055
Epoch 4/10
821/821 [==============================] - 11s 13ms/step - loss: 0.0050
Epoch 5/10
821/821 [==============================] - 11s 13ms/step - loss: 0.0048
Epoch 6/10
821/82

Total params: 198273 (774.50 KB)
Trainable params: 198273 (774.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
509/509 [==============================] - 11s 13ms/step - loss: 0.0162
Epoch 2/10
509/509 [==============================] - 7s 13ms/step - loss: 0.0113
Epoch 3/10
509/509 [==============================] - 7s 14ms/step - loss: 0.0098
Epoch 4/10
509/509 [==============================] - 7s 14ms/step - loss: 0.0089
Epoch 5/10
509/509 [==============================] - 7s 14ms/step - loss: 0.0085
Epoch 6/10
509/509 [==============================] - 7s 14ms/step - loss: 0.0083
Epoch 7/10
509/509 [==============================] - 7s 14ms/step - loss: 0.0083
Epoch 8/10
509/509 [==============================] - 7s 14ms/step - loss: 0.0081
Epoch 9/10
509/509 [==============================] - 7s 14ms/step - loss: 0.0080
Epoch 10/10
509/509 [==============================] - 7s 14ms/step - loss: 0.0078
Model:

821/821 [==============================] - 16s 20ms/step - loss: 0.0085
Epoch 4/10
821/821 [==============================] - 15s 19ms/step - loss: 0.0081
Epoch 5/10
821/821 [==============================] - 15s 18ms/step - loss: 0.0079
Epoch 6/10
821/821 [==============================] - 15s 18ms/step - loss: 0.0076
Epoch 7/10
821/821 [==============================] - 15s 18ms/step - loss: 0.0072
Epoch 8/10
821/821 [==============================] - 15s 18ms/step - loss: 0.0068
Epoch 9/10
821/821 [==============================] - 15s 18ms/step - loss: 0.0066
Epoch 10/10
821/821 [==============================] - 15s 18ms/step - loss: 0.0062
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, None, 128)         66560     
                                                                 
 dropout_28 (Dropout)        (None, None, 128)         0   

821/821 [==============================] - 12s 15ms/step - loss: 0.0097
Epoch 9/10
821/821 [==============================] - 12s 15ms/step - loss: 0.0097
Epoch 10/10
821/821 [==============================] - 12s 14ms/step - loss: 0.0096
Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_36 (LSTM)              (None, None, 128)         66560     
                                                                 
 dropout_36 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_37 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_37 (Dropout)        (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 1)                 129       
            